# Introduction to the cldrive API

In [1]:
import cldrive
cldrive.__version__

'0.0.3.dev1'

## The OpenCL Environment

A list of available platforms and devices is avaialble using `clinfo()`:

In [2]:
cldrive.clinfo()

Host: Ubuntu 16.04 64bit
Platform 0: NVIDIA CUDA
    Device 0: GPU GeForce GTX 1080 375.39
    Device 1: GPU GeForce GTX 1080 375.39
Platform 1: Intel(R) OpenCL
    Device 0: CPU Intel(R) Xeon(R) CPU E5-2620 v4 @ 2.10GHz 1.2.0.25


Construct an environment using:

In [3]:
env = cldrive.make_env()
env

Device: GeForce GTX 1080, Platform: NVIDIA Corporation

In [4]:
cldrive.OpenCLEnvironment(platform="NVIDIA CUDA", device="GeForce GTX 1080")

Device: GeForce GTX 1080, Platform: NVIDIA CUDA

In [5]:
try:
    cldrive.make_env(platform="not a real OpenCL platform")
except LookupError:
    print("no suitable environment found")

no suitable environment found


In [6]:
for devtype in ['cpu', 'gpu', 'foobar']:
    try:
        cldrive.make_env(devtype=devtype)
        print(f'environment available for device type {devtype}')
    except LookupError:
        print(f'system has no {devtype}')
    except ValueError as e:
        print(e)

environment available for device type cpu
environment available for device type gpu
unrecognized device type 'foobar'


## The Kernel Driver

In [7]:
import numpy as np

src = """\
    kernel void A(global int* data) {
        int tid = get_global_id(0);
        data[tid] *= 2;
    }
"""

inputs = [np.arange(16)]

outputs = cldrive.drive(env, src, inputs, gsize=(16,1,1), lsize=(8,1,1))
outputs

array([[ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]], dtype=int32)

In [8]:
# check that outputs are correct:
for x, y in zip(inputs[0], outputs[0]):
    if y != x * 2:
        print(f"{y} != {x} * 2")

Use keyword argument `debug` for more verbose output:

In [9]:
src = """
    kernel void A(global int* a, const int2 b) {
        const int tid = get_global_id(0);
        if (tid < get_global_size(0))
            a[tid] = a[tid] * 2.0 + b.x + b.y;
    }
"""

inputs = [[0, 1, 2, 3], [10, 11]]

outputs = cldrive.drive(env, src, inputs, gsize=(4, 1, 1), lsize=(1, 1, 1),
                        debug=True, timeout=60, optimizations=True, profiling=True)
outputs

[cldrive] Porcelain invocation: timeout --signal=9 60 /home/cec/clgen/bin/python3 /home/cec/clgen/lib/python3.6/site-packages/cldrive-0.0.3.dev1-py3.6.egg/cldrive/driver.py /tmp/cldrive-rn_1puc1.job

[cldrive] Platform: NVIDIA Corporation
[cldrive] Device: GeForce GTX 1080
[cldrive] 3-D global size 4 = [4, 1, 1]
[cldrive] 3-D local size 1 = [1, 1, 1]
[cldrive] Number of kernel arguments: 2
[cldrive] Kernel arguments: global int a*, const int2 b
[cldrive] Kernel input sizes: [4, 2]
[cldrive] OpenCL optimizations: on
[cldrive] OpenCL profiling: on
[cldrive] Compilation succeeded
[cldrive] Command queue flushed
[cldrive] Device memory allocated
[cldrive] Set kernel arguments
[cldrive] Host -> Device transfers time: 0.000672 ms
[cldrive] Kernel execution enqueued
[cldrive] Kernel execution time: 0.012544 ms
[cldrive] Device -> Host transfers time: 0.012544 ms
[cldrive] Command queue flushed
[cldrive] Porcelain subprocess complete
[cldrive] Porcelain return code: 0


array([array([21, 23, 25, 27], dtype=int32), array([10, 11], dtype=int32)], dtype=object)

In [10]:
# check that outputs are correct:
for x, y in zip(inputs[0], outputs[0]):
    if y != x * 2 + 10 + 11:
        print(f"{y} != {x} * 2")